In [ ]:
## Code without tflite converter

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the data
data = pd.read_csv('imu_data.csv')

# Handle missing values if any
data = data.dropna()

# Separate features (Roll, Pitch) and target (Direction)
X = data[['Roll', 'Pitch']].values
y = data['Direction'].values

# Encode the target variable (Direction)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # Converts labels to numbers

# Scale the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use this for integer labels
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

# Function to predict direction
def predict_direction(roll, pitch):
    # Scale the input
    input_data = scaler.transform(np.array([[roll, pitch]]))

    # Make a prediction
    prediction = model.predict(input_data)

    # Get the predicted class
    predicted_class = np.argmax(prediction)

    # Decode the prediction
    predicted_direction = label_encoder.inverse_transform([predicted_class])[0]

    return predicted_direction

# Example usage
roll_input = float(input("Enter Roll value: "))
pitch_input = float(input("Enter Pitch value: "))

predicted_direction = predict_direction(roll_input, pitch_input)
print(f"Predicted Direction: {predicted_direction}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 124ms/step - accuracy: 0.6674 - loss: 1.0039 - val_accuracy: 0.8451 - val_loss: 0.7371
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8552 - loss: 0.6992 - val_accuracy: 0.9718 - val_loss: 0.4447
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9272 - loss: 0.4432 - val_accuracy: 0.9859 - val_loss: 0.2452
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9634 - loss: 0.2443 - val_accuracy: 0.9859 - val_loss: 0.1445
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9758 - loss: 0.1397 - val_accuracy: 0.9859 - val_loss: 0.0939
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9656 - loss: 0.1115 - val_accuracy: 0.9859 - val_loss: 0.0748
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9635 - loss: 0.0911 - val_accuracy: 0.9859 - val_loss: 0.0604
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9802 - loss: 0.0658 - val_accuracy: 1.0000 - val_los

In [3]:
## Code with tflite converter

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the data
data = pd.read_csv('imu_data.csv')

# Handle missing values if any
data = data.dropna()

# Separate features (Roll, Pitch) and target (Direction)
X = data[['Roll', 'Pitch']].values
y = data['Direction'].values

# Encode the target variable (Direction)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # Converts labels to numbers

# Scale the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use this for integer labels
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Convert the model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Optional: Post-training quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Provide a representative dataset for quantization (required for some quantization types)
def representative_data_gen():
    for i in range(100):  # Use the first 100 samples
        yield [X_train[i].astype(np.float32)]
converter.representative_dataset = representative_data_gen

# Convert to integer-only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # tf.float32 or tf.uint8
converter.inference_output_type = tf.int8  # tf.float32 or tf.uint8

tflite_model = converter.convert()

# Save the TFLite model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("TFLite model converted and saved as model.tflite")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step - accuracy: 0.6188 - loss: 0.9846 - val_accuracy: 0.9437 - val_loss: 0.6722
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8771 - loss: 0.6456 - val_accuracy: 0.9718 - val_loss: 0.3905
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9466 - loss: 0.3798 - val_accuracy: 0.9859 - val_loss: 0.2181
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9607 - loss: 0.2184 - val_accuracy: 0.9859 - val_loss: 0.1327
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9685 - loss: 0.1371 - val_accuracy: 0.9859 - val_loss: 0.0903
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9762 - loss: 0.1072 - val_accuracy: 1.0000 - val_loss: 0.0695
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9576 - loss: 0.0975 - val_accuracy: 1.0000 - val_loss: 0.0624
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9628 - loss: 0.0931 - val_accuracy: 1.0000 - val_loss: 0.0505
Ep

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
